In [1]:
import time
import numpy as np
import pandas as pd
from urllib.request import Request, urlopen
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import urllib
import tqdm
import warnings
warnings.simplefilter(action="ignore",category=FutureWarning)

chrome_options = webdriver.ChromeOptions()

In [2]:
url = "https://www.saramin.co.kr/zf_user/"

In [3]:
driver = webdriver.Chrome(service = Service('data_analysis/Driver/chromedriver'),options=chrome_options)

In [ ]:
# driver = webdriver.Chrome(executable_path="C:/Users/leeyj/LYJPython/Driver/chromedriver.exe")

In [4]:
driver.get(url)

In [5]:
driver.find_element(By.ID, "btn_search" ).click()

In [6]:
search = driver.find_element(By.ID, "ipt_keyword_recruit")

In [7]:
search.send_keys("로봇")

In [8]:
search.send_keys(Keys.ENTER)

In [9]:
driver.find_element(By.CSS_SELECTOR,"#recruit_info_list > div.content_bottom > div > a").click()

In [10]:
driver.find_element(By.XPATH,'''//*[@id="recruit_info_list"]/div[2]/div/a[1]''').click()

In [11]:
name_list = []    # 회사명
url_list = []    # 공고 URL
title_list = []    # 공고 제목
region_list = []    # 근무지
exp_list = []    # 경력 
degree_list = []    # 학력
position_list = []    # 고용 형태
salary_list = []    # 연봉
address_list = []    # 상세 주소
welfare_list = []    # 사내 복지 
statistics_list = []    # 지원자 통계
comInfo_list = []    # 기업 정보
detail_list = []    # 공고 내 html 세부정보
tag_list = []    # 태그

In [13]:
def find_info():
    # 페이지 내 모든 공고문들이 들어있는 contents 변수 설정
    contents = driver.find_element(By.CLASS_NAME,"content").find_elements(By.CLASS_NAME,"item_recruit")

    for i in range(len(contents)):    # 공고들의 갯수만큼 반복
        items = contents[i].find_element(By.CLASS_NAME,"job_condition").find_elements(By.TAG_NAME,"span")
        if len(items) == 3:
                name_list.append(contents[i].find_element(By.CLASS_NAME,"corp_name").text)    # 회사명
                url_list.append(contents[i].find_element(By.CLASS_NAME,"job_tit").find_element(By.TAG_NAME,'a').get_attribute("href")) # 공고 URL
                title_list.append(contents[i].find_element(By.CLASS_NAME,"job_tit").text)    # 공고 제목
                region_list.append(items[0].text)    # 근무지
                exp_list.append(items[1].text)    # 경력
                degree_list.append(items[2].text)    # 학력
                position_list.append("")    # 고용 형태
                salary_list.append("")    # 연봉

        elif len(items) == 4:
                name_list.append(contents[i].find_element(By.CLASS_NAME,"corp_name").text)    
                url_list.append(contents[i].find_element(By.CLASS_NAME,"job_tit").find_element(By.TAG_NAME,'a').get_attribute("href")) 
                title_list.append(contents[i].find_element(By.CLASS_NAME,"job_tit").text)    
                region_list.append(items[0].text)
                exp_list.append(items[1].text)
                degree_list.append(items[2].text)
                position_list.append(items[3].text)
                salary_list.append("")

        elif len(items) == 5:
                name_list.append(contents[i].find_element(By.CLASS_NAME,"corp_name").text)    
                url_list.append(contents[i].find_element(By.CLASS_NAME,"job_tit").find_element(By.TAG_NAME,'a').get_attribute("href")) 
                title_list.append(contents[i].find_element(By.CLASS_NAME,"job_tit").text)    
                region_list.append(items[0].text)
                exp_list.append(items[1].text)
                degree_list.append(items[2].text)
                position_list.append(items[3].text)
                salary_list.append(items[4].text)

        ### ---------- 공고클릭시 나오는 ------- ###
        ### ---------- 세부정보 저장 ---------- ###
        contents[i].find_element(By.TAG_NAME,"a").click()    
        driver.implicitly_wait(1)
        second_tab = driver.window_handles[1]   
        driver.switch_to.window(window_name=second_tab)    # 드라이버 포커스를 새 탭(페이지)으로 이동
        # 상세 주소
        try:  
            address_list.append(driver.find_element(By.CLASS_NAME,"address").text)    
        except:
            address_list.append("")
        # 사내 복지
        try:
            # 복리후생이 공고에 포함되있을 때 복지 더보기가 클릭 가능하다면 클릭하여 크롤링
            if driver.find_element(By.CLASS_NAME,"jv_cont.jv_benefit").find_element(By.CLASS_NAME,"jv_title").text == "복리후생":
                if driver.find_element(By.CLASS_NAME,"jv_cont.jv_benefit").find_element(By.CLASS_NAME,"btn_more_cont").click():
                     welfare_list.append(driver.find_element(By.CLASS_NAME,"jv_cont.jv_benefit").find_element(By.CLASS_NAME,"details").text) 
        except:
            pass
        try:
             # 복리후생이 공고에 포함되있을 때 복지 더보기가 없다면 그냥 크롤링
             if driver.find_element(By.CLASS_NAME,"jv_cont.jv_benefit").find_element(By.CLASS_NAME,"jv_title").text == "복리후생":
                  welfare_list.append(driver.find_element(By.CLASS_NAME,"jv_cont.jv_benefit").find_element(By.CLASS_NAME,"details").text)
        except:
             # 위의 도 조건에 부합하므로 빈 문자열을 사내 복지 리스트에 저장
             welfare_list.append("")
        # 지원자 통계
        ###---------- 지원자 통계를 보기위해 로그인 필요 ---------- ###
        try:
            driver.find_element(By.CLASS_NAME,"jv_cont.jv_statics").find_element(By.CLASS_NAME,"btn_more_cont").click() 
            statistics_list.append(driver.find_element(By.CLASS_NAME,"jv_cont.jv_statics.expand").find_element(By.CLASS_NAME,"details").text)
        except:
            statistics_list.append("")
        # 기업 정보
        try:
            comInfo_list.append(driver.find_element(By.CLASS_NAME,"jv_cont.jv_company").find_element(By.CLASS_NAME,"info").text) 
        except:
            comInfo_list.append("")
        # 태그
        try:
            tmp = []
            driver.find_element(By.CLASS_NAME,"jv_cont.jv_footer").find_element(By.CLASS_NAME,"tags").find_element(By.TAG_NAME,"button").click()
            tags = driver.find_element(By.CLASS_NAME,"jv_cont.jv_footer").find_element(By.CLASS_NAME,"tags.on").find_elements(By.TAG_NAME,"li")
            for i in range(len(tags)):
                tmp.append(tags[i].text)
            tag_list.append(tmp)
        except:
            tmp = []
            tags = driver.find_element(By.CLASS_NAME,"jv_cont.jv_footer").find_element(By.CLASS_NAME,"tags").find_elements(By.TAG_NAME,"li")
            for i in range(len(tags)):
                tmp.append(tags[i].text)
            tag_list.append(tmp)

        # 채용 공고 html 이동
        try:
            # 공고 html로 드라이버 포커스를 이동
            driver.switch_to.frame(driver.find_element(By.NAME,'iframe_content_0'))
            if len(driver.find_element(By.CLASS_NAME,"user_content").text )>=1 == True:
                print("텍스트 공고")
                detail_list.append(driver.find_element(By.CLASS_NAME,"user_content").text)
            else:
                print("사진 공고")
                detail_list.append("사진")
        except:
            print("크롤링 불가")    # 반복 실행 불가를 일으키는 오류는 pass
            detail_list.append("")
            
        driver.close()
        first_tab = driver.window_handles[0]
        driver.switch_to.window(window_name=first_tab)

In [14]:
find_info()

텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고
사진 공고
텍스트 공고
텍스트 공고
텍스트 공고
텍스트 공고


### 1~10페이지 저장

In [ ]:
for k in range(len(driver.find_element(By.CLASS_NAME,"pagination").find_elements(By.TAG_NAME,"span"))):
    find_info()
    if k == len(driver.find_element(By.CLASS_NAME,"pagination").find_elements(By.TAG_NAME,"span")):
        driver.find_element(By.CSS_SELECTOR,"#recruit_info_list > div.content_bottom > div > a.btnNext.page_move.track_event").click()
    j = k+1    
    cnt_str = '''//*[@id="recruit_info_list"]/div[2]/div/a[''' + str(j) + "]"
    driver.find_element(By.XPATH,cnt_str).click()
    time.sleep(0.5)

### 11페이지 후부터 10 페이지 단위로 크롤링(11~20)

In [ ]:
# for k in range(len(driver.find_element(By.CLASS_NAME,"pagination").find_elements(By.TAG_NAME,"span"))):
#     find_info()
#     if k == len(driver.find_element(By.CLASS_NAME,"pagination").find_elements(By.TAG_NAME,"span")):
#         driver.find_element(By.CSS_SELECTOR,"#recruit_info_list > div.content_bottom > div > a.btnNext.page_move.track_event").click()
#         break
#     j = k+2    
#     cnt_str = '''//*[@id="recruit_info_list"]/div[2]/div/a[''' + str(j) + "]"
#     driver.find_element(By.XPATH,cnt_str).click()
#     time.sleep(0.5)

### 11페이지 후부터 "다음" 버튼이 존재하는 페이지 까지 반복하며 계속 크롤링

In [ ]:
while True:
    # 다음 버튼을 계속 클릭할 수 있는 경우 무한 반복하며 크롤링
    # 다음 반복 여부를 flag에 저장
    # 다음 버튼이 존재한다면 for문을 돌며 1~10페이지등의 반복을 수행
    flag = len(driver.find_element(By.CSS_SELECTOR,"#recruit_info_list > div.content_bottom > div > a.btnNext.page_move.track_event").text)
    if flag>=0:
        for k in range(len(driver.find_element(By.CLASS_NAME,"pagination").find_elements(By.TAG_NAME,"span"))):
            find_info()    # 한 페이지 내 크롤링
            if k == len(driver.find_element(By.CLASS_NAME,"pagination").find_elements(By.TAG_NAME,"span")):    
                # 페이지의 끝에 도달할 경우
                # 다음 버튼을 클릭후 탈출하여 다시 flag를 검사
                driver.find_element(By.CSS_SELECTOR,"#recruit_info_list > div.content_bottom > div > a.btnNext.page_move.track_event").click()
                break
            j = k+2    
            cnt_str = '''//*[@id="recruit_info_list"]/div[2]/div/a[''' + str(j) + "]"
            driver.find_element(By.XPATH,cnt_str).click()
            time.sleep(0.5)
    else:
        break    # 다음 버튼이 없으면 반복문 탈출

In [15]:
# 디버깅용 리스트 길이 확인
print(len(name_list), len(title_list), len(url_list), len(region_list), len(exp_list), len(degree_list), len(position_list), len(detail_list), len(comInfo_list), len(welfare_list), len(statistics_list), len(tag_list))

40 40 40 40 40 40 40 40 40 40 40 40


In [17]:
# 2차원 리스트인 tag_list를 1차원 형태로
# tag_list의 # 버리기
# for i in range(len(detail_list)):
#     detail_list[i] = detail_list[i].strip()
for i in range(len(tag_list)):
    tag_list[i] = ','.join(tag_list[i])
    tag_list[i] = str(tag_list[i]).replace("#","",True)

#### 데이터 프레임으로 만든 후 저장

In [43]:
saramin_df = pd.DataFrame({"회사명":name_list,"직무":title_list,"URL":url_list, "근무지":region_list,"경력":exp_list,\
    "학위":degree_list,"유형":position_list,"사내복지":welfare_list, "상세 주소":address_list, "기업 정보":comInfo_list, "지원자 통계":statistics_list, "태그":tag_list, "세부정보":detail_list})

In [47]:
saramin_df.to_excel("../data/saramin_df.xlsx",engine="xlsxwriter")
saramin_df.to_csv("../data/saramin_df.csv",sep=",",encoding="utf-8")

In [48]:
driver.close()